이 노트북은 Chat Completion API와 외부 함수들을 결합하여 GPT 모델의 기능을 확장하는 방법을 다룹니다.

tools는 Chat Completion API의 선택적 매개변수로, 함수 명세(function specifications)를 제공하는 데 사용될 수 있습니다. 이를 통해 모델이 제공된 명세에 맞는 함수 인자를 생성할 수 있습니다. 그러나 API는 실제로 함수 호출을 실행하지 않습니다. 함수 호출을 실행하는 것은 개발자의 책임입니다.

tools 매개변수 안에 functions가 제공되면, 기본적으로 모델이 함수 사용 여부를 스스로 결정합니다. 특정 함수를 강제로 사용하려면 tool_choice 매개변수를 {"type": "function", "function": {"name": "my_function"}}로 설정합니다. 반대로 함수를 사용하지 않도록 강제하려면 tool_choice 매개변수를 "none"으로 설정합니다. 함수가 사용되면 응답에 "finish_reason": "tool_calls"와 함께 함수 이름과 생성된 함수 인자를 포함하는 tool_calls 객체가 나타납니다.

**개요**

이 노트북에는 두 가지 주요 섹션이 있습니다:

- 함수 인자를 생성하는 방법: 함수 세트를 지정하고 API를 사용하여 함수 인자를 생성하는 방법을 다룹니다.
- 모델이 생성한 인자를 사용하여 함수를 호출하는 방법: 모델이 생성한 인자를 실제로 사용하여 함수를 호출하는 방법을 설명합니다.

이 설명을 통해 Chat Completion API와 외부 함수 결합에 대한 개념을 알 수 있습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## How to generate function arguments

!pip install scipy --quiet

설치되는 패키지: scipy
- SciPy는 과학 계산을 위한 오픈 소스 Python 라이브러리입니다. 이 라이브러리는 고급 수학, 과학 및 공학 문제를 해결하기 위한 많은 함수와 도구를 제공합니다.

설치되는 패키지: tenacity
- Tenacity는 함수 재시도를 관리하기 위한 Python 라이브러리입니다. 네트워크 요청, 데이터베이스 연결 등 실패할 수 있는 작업을 재시도하는 로직을 쉽게 구현할 수 있게 도와줍니다.

설치되는 패키지: tiktoken
- Tiktoken은 주로 자연어 처리를 위한 토크나이저입니다. 이 패키지는 텍스트를 더 작은 단위인 토큰으로 분할하는 기능을 제공합니다.

설치되는 패키지: openai
- OpenAI는 OpenAI API를 사용하기 위한 Python 클라이언트 라이브러리입니다. 이를 통해 OpenAI의 GPT 모델과 같은 다양한 인공지능 기능을 사용할 수 있습니다.


In [3]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken --quiet
!pip install openai --quiet

In [4]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

In [7]:
# @retry: 이 데코레이터는 함수가 실패할 경우 자동으로 재시도하도록 합니다.
# wait=wait_random_exponential(multiplier=1, max=40): 재시도 사이의 대기 시간을 설정합니다. 대기 시간은 랜덤 지수 증가 방식으로, 최대 40초까지 증가합니다.
# stop=stop_after_attempt(3): 최대 3번까지 재시도합니다.
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


### Basic concepts

가상의 날씨 API와 연동하기 위해 함수 명세(function specifications)를 만들어 보겠습니다. 이 함수 명세를 Chat Completions API에 전달하여 해당 명세를 준수하는 함수 인자를 생성하도록 할 것입니다.

즉, 함수 명세를 정의하고 이를 API에 넘겨주면, API는 그 명세에 맞는 입력값을 자동으로 생성하게 됩니다. 이 과정에서, 날씨 API와의 인터페이스가 올바르게 작동하도록 적절한 인자를 모델이 자동으로 만들 수 있게 도와주는 것입니다.

In [5]:
# 두 개의 함수형 도구를 정의하여 날씨 정보를 가져오는 기능
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

우리가 모델에 현재 날씨에 대한 질문을 던지면, 모델은 몇 가지 명확한 질문을 던질 것입니다.

In [8]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message.content


'Could you please provide me with your city and state so I can give you the current weather?'

누락된 정보를 제공하면 해당 함수 인수가 자동으로 생성됩니다.

In [9]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message.content


'Would you like the temperature in Celsius or Fahrenheit?'

다른 방식으로 프롬프트를 표시하면, 우리가 알려준 다른 기능을 타겟으로 지정할 수 있습니다.

In [10]:
# 사용자가 글래스고(Glasgow), 스코틀랜드의 특정 기간(x일) 동안의 날씨를 묻는 요청을 챗봇에게 전달하고, 챗봇의 응답을 메시지 목록에 추가하는 과정을 구현
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message.content


'Could you please specify the number of days for the weather forecast you would like for Glasgow, Scotland? Additionally, would you prefer the temperature in Celsius or Fahrenheit?'

다시 한번, 모델은 아직 충분한 정보가 없기 때문에 우리에게 설명을 요청하고 있습니다. 이 경우 모델은 이미 예보의 위치를 ​​알고 있지만 예보에 필요한 일수는 알아야 합니다.

In [11]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0].message.content


'Would you like the temperature to be displayed in Celsius or Fahrenheit for the 5-day weather forecast in Glasgow, Scotland?'

#### Forcing the use of specific functions or no function

예를 들어, function_call 인수를 사용하여 get_n_day_weather_forecast와 같이 특정 함수를 사용하도록 모델을 강제할 수 있습니다. 이렇게 하면 모델이 함수를 사용하는 방법에 대한 가정을 하도록 강제합니다.

In [13]:
# 사용자가 토론토, 캐나다의 날씨 보고서를 요청하는 메시지를 챗봇에게 전달하고, 챗봇이 해당 요청에 따라 응답을 생성하는 과정을 구현한 것입니다.
# 여기서는 특정 도구(get_n_day_weather_forecast)를 사용하여 챗봇이 응답하도록 지정
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada for the next 3 days."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.choices[0].message.content

In [14]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0].message.content

'Would you like the current weather report, or would you prefer a multi-day forecast for Toronto? Additionally, please specify the temperature unit you would like to use: Celsius or Fahrenheit.'

우리는 또한 모델이 함수를 전혀 사용하지 않도록 강제할 수 있습니다. 그렇게 하면 적절한 함수 호출을 생성하지 못하게 됩니다.

In [15]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
chat_response.choices[0].message.content


"I'll retrieve the current weather for Toronto, Canada in Celsius. One moment, please."

### Parallel Function Calling

gpt-4o나 gpt-3.5-turbo와 같은 최신 모델은 한 번에 여러 기능을 호출할 수 있습니다.

In [16]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message.tool_calls
assistant_message

[ChatCompletionMessageToolCall(id='call_MSMj9CpFje9JH8SHU27SMAHY', function=Function(arguments='{"location": "San Francisco, CA", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'),
 ChatCompletionMessageToolCall(id='call_5EzydGvqkJzPrYyAYkKrfRuh', function=Function(arguments='{"location": "Glasgow, UK", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]

openweather 사이트 api로 연결하여 현재날씨를 조회할 수 있는 챗봇

https://home.openweathermap.org/

In [29]:
import openai
import json
import requests

# OpenAI API 키 설정
from openai import OpenAI
client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

# 날씨 API 키 설정 (예: OpenWeatherMap)
weather_api_key = '14d17402bcdf470d7b9ef78fc5658ca7'

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# 사용자 요청 메시지
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the current weather in New York?"}
]

# GPT-4 모델 호출
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    functions=[{
        "name": "get_current_weather",
        "description": "Get the current weather for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the city to get the weather for"
                }
            },
            "required": ["location"]
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다.
response_message = response.choices[0].message
messages.append(response_message)

print(response_message)

# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name == 'get_current_weather':
        location = tool_arguments['location']
        weather_results = get_current_weather(location)

        # 함수 호출 결과 메시지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_results)  # JSON 형식으로 반환
        })

        # 모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)


ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"New York"}', name='get_current_weather'), tool_calls=None)
The current weather in New York is misty. Here are the details:

- **Temperature:** 23.06°C (feels like 23.69°C)
- **Humidity:** 87%
- **Wind Speed:** 5.81 m/s coming from the southeast (124°), with gusts up to 8.05 m/s
- **Visibility:** 4,828 meters
- **Cloud Cover:** 100% (overcast)

If you need more specific information or a forecast, feel free to ask!


전체코드

#### Function Calling의 사용
- 이 코드에서 function calling은 모델이 적절한 시점에 외부 함수(get_current_weather)를 호출하도록 유도합니다. 모델이 What is the current weather in {user_location}?와 같은 질문에 응답할 때, 해당 함수가 정의되어 있으면 모델은 그 함수를 호출하여 날씨 정보를 가져오려 합니다.

- function_call="auto" 설정을 통해 GPT 모델은 적절한 함수가 있을 때 자동으로 그 함수를 호출할 수 있습니다. 여기서는 get_current_weather 함수 명세를 통해 사용자가 입력한 위치에 맞는 날씨 정보를 가져올 수 있도록 유도됩니다.

- 만약 모델이 함수 호출을 결정하면, 응답에 function_call이 포함되며, 이때 함수 이름과 그 인자가 함께 반환됩니다. 그런 후에 함수가 호출되고, 결과가 다시 모델에 제공됩니다.

이 과정에서 모델은 단순히 인자를 생성하고, 함수를 호출하는 것은 개발자의 책임입니다. OpenAI의 API는 실제 함수 실행을 수행하지 않으므로, 함수 호출 후 결과를 처리하고 다시 모델에 넘기는 과정을 수동으로 처리하게 됩니다.

In [30]:
import openai
import json
import requests

# OpenAI API 키 설정
from openai import OpenAI
client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

# 날씨 API 키 설정 (예: OpenWeatherMap)
weather_api_key = '14d17402bcdf470d7b9ef78fc5658ca7'

# 이 함수는 주어진 location에 대해 OpenWeatherMap API에서 날씨 정보를 요청하고, 응답을 JSON 형식으로 반환합니다.
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# 사용자 요청 메시지 초기화
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Please provide the location to get the current weather information."}
]

# 사용자 입력 받기
user_location = input("Enter the location to get the current weather: ")
messages.append({"role": "user", "content": f"What is the current weather in {user_location}?"})

# GPT-4 모델 호출
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    functions=[{
        "name": "get_current_weather", # 위치 정보를 받아 날씨 정보를 반환하는 함수로 정의
        "description": "Get the current weather for a specific location", # location이라는 문자열 매개변수를 요구하며, 이 매개변수를 통해 날씨 정보를 얻습니다.
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the city to get the weather for"
                }
            },
            "required": ["location"]
        }
    }],
    function_call="auto"
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다.
response_message = response.choices[0].message
messages.append(response_message)

print(response_message)

# 도구 호출 여부 확인
function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name == 'get_current_weather':
        location = tool_arguments['location']
        weather_results = get_current_weather(location)

        # 함수 호출 결과 메시지 추가
        messages.append({
            "role": "function",
            "name": tool_function_name,
            "content": json.dumps(weather_results)  # JSON 형식으로 반환
        })

        # 모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)


Enter the location to get the current weather: New York
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"New York"}', name='get_current_weather'), tool_calls=None)
The current weather in New York is as follows:

- **Condition**: Mist
- **Temperature**: 23.17°C
- **Feels Like**: 23.79°C
- **Humidity**: 86%
- **Wind Speed**: 5.81 m/s (from the southeast)
- **Visibility**: 10,000 meters
- **Cloud Cover**: 100%

Overall, it's a misty day with high humidity.


## How to call functions with model generated arguments

다음 예제에서는 모델에서 생성된 입력을 갖는 함수를 실행하는 방법을 보여주고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현합니다. 단순화를 위해 Chinook 샘플 데이터베이스를 사용하겠습니다 .

참고: 모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서 SQL 생성은 위험할 수 있습니다.

이 코드는 OpenAI의 GPT 모델을 사용하여 SQLite 데이터베이스에서 음악 관련 질문에 대한 답변을 SQL 쿼리로 변환한 후, 해당 쿼리를 실행하여 결과를 반환하는 방식으로 function calling을 구현한 예시입니다. 여기서는 사용자가 앨범 관련 질문을 하면, GPT-4가 질문을 SQL 쿼리로 변환하고, SQLite 데이터베이스를 조회하여 결과를 반환합니다.

### Specifying a function to execute SQL queries

먼저 SQLite 데이터베이스에서 데이터를 추출하는 데 유용한 유틸리티 함수를 정의해 보겠습니다.

Chinook.db 데이터베이스에 연결하여 SQLite를 통해 SQL 쿼리를 실행할 수 있도록 합니다.

In [19]:
import sqlite3
from google.colab import drive

drive.mount('/content/drive')
conn = sqlite3.connect("/content/drive/MyDrive/kdt_240424/m9_openai/data/Chinook.db")
print("Opened database successfully")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Opened database successfully


데이터베이스 테이블 및 열 정보 조회

- get_table_names, get_column_names 함수를 통해 데이터베이스의 테이블 및 열 이름을 가져옵니다.
- 이 정보는 나중에 GPT 모델이 SQL 쿼리를 생성할 때 사용할 스키마 정보를 제공하는 데 활용됩니다.

In [20]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


데이터베이스의 테이블과 열 정보를 문자열 형태로 저장하여, 나중에 SQL 쿼리 작성 시 참조할 수 있도록 합니다.

In [21]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

tools라는 리스트에 ask_database라는 함수 명세를 정의합니다.
- 이 함수는 사용자의 질문에 맞는 SQL 쿼리를 받아 데이터베이스에서 정보를 조회하고, 이를 반환하는 기능을 합니다.
- 함수의 매개변수로 query가 있으며, 이는 SQL 쿼리를 텍스트 형태로 전달받아 실행하는 구조입니다.
- 중요한 점은 database_schema_string이 함수 설명에 포함되어 있어, 모델이 데이터베이스 스키마에 맞는 SQL 쿼리를 생성할 수 있도록 도움을 줍니다.

In [22]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

### Executing SQL queries

이제 실제로 데이터베이스에 대한 쿼리를 실행하는 함수를 구현해 보겠습니다.

In [23]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

##### Steps to invoke a function call using Chat Completions API:

- 1단계 : 모델이 사용할 도구를 선택하도록 하는 내용으로 모델을 프롬프트합니다. 함수 이름 및 서명과 같은 도구에 대한 설명은 '도구' 목록에 정의되어 API 호출에서 모델에 전달됩니다. 선택한 경우 함수 이름과 매개변수가 응답에 포함됩니다.
- 2단계 : 모델이 함수를 호출하려고 하는지 프로그래밍적으로 확인합니다. 참이면 3단계로 진행합니다.
- 3단계 : 응답에서 함수 이름과 매개변수를 추출하고 매개변수와 함께 함수를 호출합니다. 결과를 메시지에 추가합니다.
- 4단계 : 메시지 목록으로 채팅 완료 API를 호출하여 응답을 가져옵니다.

In [24]:
# 사용자의 요청 메시지를 정의합니다.
messages = [{
    "role":"user",
    "content": "What is the name of the album with the most tracks?"
}]

# 사용자의 질문에 대한 응답을 생성. tools와 tool_choice 파라미터는 모델이 데이터베이스 쿼리를 인식하고 자동으로 도구를 선택할 수 있도록 돕습니다.
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    tools= tools,
    tool_choice="auto" # 모델이 자동으로 함수 호출을 결정S, QL 쿼리로 변환할 필요가 있다고 판단하면 자동으로 ask_database 함수를 호출하게 됩니다.
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력합니다.
response_message = response.choices[0].message
messages.append(response_message)

print(response_message.content)

None


In [25]:
# 모델 응답에서 도구 호출이 포함되어 있는지 확인하고, 도구 호출이 있다면 도구 호출 ID, 함수 이름 및 쿼리 문자열을 추출합니다.
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)['query']

    # 도구 호출 함수 이름이 'ask_database'인 경우, ask_database 함수를 호출하여 데이터베이스 쿼리를 실행하고 결과를 messages 리스트에 추가합니다.
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)

        messages.append({
            "role":"tool",
            "tool_call_id":tool_call_id,
            "name": tool_function_name,
            "content":results
        })

        # 도구 호출 결과가 포함된 messages 리스트를 사용하여 모델을 다시 호출하고 최종 응답을 출력합니다.
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
        # 도구 호출 함수 이름이 'ask_database'가 아닌 경우 오류 메시지를 출력하거나, 도구 호출이 없으면 모델의 응답 내용을 바로 출력합니다.
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # Model did not identify a function to call, result can be returned to the user
    print(response_message.content)

The album with the most tracks is titled "Greatest Hits".
